Part 1

In [1]:
import bisect   # binary search library

# define mapping class
class Map:
    def __init__(self, data):
        self.src, _, self.dest = data[0].split()[0].split('-')
        self.lines = [[int(x) for x in line.split()] for line in data[1:]]
        self.lines.sort(key = lambda x: x[1])
        self.dests, self.srcs, self.ns = zip(*self.lines)
        self.ranges = len(self.ns)

    def __str__(self):
        return f'''{self.src} -> {self.dest}
srcs  = {self.srcs}
dests = {self.dests}
ns    = {self.ns}
ranges= {self.ranges}'''
    
    def search(self, key, reverse=False):
        if reverse:
            self.lines.sort(key = lambda x: x[0])
            srcs, dests, ns = zip(*self.lines)
        else:
            srcs, dests, ns = self.srcs, self.dests, self.ns
        return self.bisect(key, srcs, dests, ns)
        
    def bisect(self, key, srcs, dests, ns):
        i = bisect.bisect(srcs, key) - 1
        if srcs[i] <= key < (srcs[i] + ns[i]):
            return dests[i] + (key - srcs[i])
        else:
            return key

In [2]:
def pt1(filename):
    with open(filename, 'r') as f:
        data = f.read().splitlines()

    seeds = [int(x) for x in data[0].split(':')[1].strip().split()]

    # build maps from input data, assumed to be sequential
    maps = []
    while data := data[data.index('') + 1:]:
        if '' in data:
            maps.append(Map(data[:data.index('')]))
        else:
            maps.append(Map(data))
            break

    # # map search tests
    # for x in [0, 1, 10, 48, 49, 50, 51, 53, 96, 97, 98, 99, 100]:
    #     print(x, maps[0].search(x))
    # for x in [79, 14, 55, 13]:
    #     print(x, maps[0].search(x))

    locations = []
    for seed in seeds:
        x = seed
        for map in maps:
            x = map.search(x)
        locations.append(x)
    return min(locations)

pt1('test.txt'), pt1('input.txt')

(35, 3374647)

Part 2

In [3]:
def pt2(filename):
    with open(filename, 'r') as f:
        data = f.read().splitlines()

    seednums = [int(x) for x in data[0].split(':')[1].strip().split()]
    seedrngs = [
        range(seednums[i], seednums[i] + seednums[i+1])
        for i in range(0, len(seednums), 2)
    ]

    maps = []
    while data := data[data.index('') + 1:]:
        if '' in data:
            maps.append(Map(data[:data.index('')]))
        else:
            maps.append(Map(data))
            break
    maps = maps[::-1]

    for start in sorted(maps[0].dests):
        i = maps[0].dests.index(start)
        for loc in range(start, start + maps[0].ns[i]):
            x = loc
            for map in maps:
                x = map.search(x, reverse=True)
                
            if any(x in rng for rng in seedrngs):
                return loc

pt2('test.txt'), pt2('input.txt')

(56, 6082852)